In [ ]:
### Import Libraries.

import scanpy as sc
import pandas as pd
import numpy as np
import pyscenic
from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.utils import df2regulons
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multitest import multipletests
import networkx as nx
import re
from tqdm import tqdm

In [ ]:
sc.settings.njobs = 40

In [ ]:
# Folder Structure.

RESOURCES_FOLDERNAME = "/resources_folder/"
AUXILLIARIES_FOLDERNAME = "/auxilliaries_folder/"
RESULTS_FOLDERNAME = "/results_folder/"
RESULTS_ROUNDS_FOLDERNAME = "/results_folder/"
FIGURES_FOLDERNAME = "/figures_folder/"

In [ ]:
### Auxilliaries.

HUMAN_TFS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'TF_list.txt')
RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join(AUXILLIARIES_FOLDERNAME, fn),
                       ['hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather',
                        'hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather']))
MOTIF_ANNOTATIONS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')

In [ ]:
### Set File Annotation.

DATASET_ID = "MC_SC_Oligos_SCENIC"
#TCGA_CODE = 'Whole_Dataset'

In [ ]:
sc.settings.figdir = FIGURES_FOLDERNAME

In [ ]:
# Auxilliary Functions.

BASE_URL = "https://motifcollections.aertslab.org/v10nr_clust/logos/"
COLUMN_NAME_LOGO = "MotifLogo"
COLUMN_NAME_MOTIF_ID = "MotifID"
COLUMN_NAME_TARGETS = "TargetGenes"

def savesvg(fname: str, fig, folder: str = FIGURES_FOLDERNAME) -> None:
    """
    Save figure as vector-based SVG image format.
    """
    fig.tight_layout()
    fig.savefig(os.path.join(folder, fname), format =' svg')
    

    
def display_logos(df: pd.DataFrame, top_target_genes: int = 3, base_url: str = BASE_URL):
    """
    :param df:
    :param base_url:
    """
    df = df.copy()
    
    def create_url(motif_id):
        return '<img src="{}{}.png" style="max-height:124px;"></img>'.format(base_url, motif_id)
    df[("Enrichment", COLUMN_NAME_LOGO)] = list(map(create_url, df.index.get_level_values(COLUMN_NAME_MOTIF_ID)))
    
    def truncate(col_val):
        return sorted(col_val, key=op.itemgetter(1))[:top_target_genes]
    df[("Enrichment", COLUMN_NAME_TARGETS)] = list(map(truncate, df[("Enrichment", COLUMN_NAME_TARGETS)]))
    
    MAX_COL_WIDTH = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    display(HTML(df.head().to_html(escape = False)))
    pd.set_option('display.max_colwidth', MAX_COL_WIDTH)

In [ ]:
### Create Results .

METADATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}_metadata.csv'.format(DATASET_ID))
EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}_QC_tpm.csv'.format(DATASET_ID))
ADJACENCIES_FNAME = os.path.join(RESULTS_ROUNDS_FOLDERNAME, '{}_Adjacencies.tsv'.format(DATASET_ID))
MOTIFS_FNAME = os.path.join(RESULTS_ROUNDS_FOLDERNAME, '{}_Regulon.csv'.format(DATASET_ID))
REGULONS_DAT_FNAME = os.path.join(RESULTS_ROUNDS_FOLDERNAME, '{}_Regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_ROUNDS_FOLDERNAME, '{}_auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_ROUNDS_FOLDERNAME, '{}_bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_ROUNDS_FOLDERNAME, '{}_thresholds.csv'.format(DATASET_ID))
ANNDATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad'.format(DATASET_ID))
LOOM_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.loom'.format(DATASET_ID))

In [ ]:
### Load Data.

os.chdir("/folder/")
adata = sc.read_h5ad("adata.h5ad")

In [ ]:
### Filter TFs
tf_df = pd.read_csv("TF_list.csv")
top_tf_threshold = 0.25
tf_counts = adata[:, tf_df['TFs']].X.sum(axis = 0)
tf_keep = tf_df['TFs'][tf_counts >= np.percentile(tf_counts, 100 * top_tf_threshold)]

In [ ]:
### Save Filtered TF List.

os.chdir("/auxilliaries/")
with open("TFs_top_25pc.txt", "w") as f:
    for tf in tf_keep:
        f.write(tf + "\n")

print(f"Saved {len(tf_keep)} TFs to TFs_top_25pc.txt")

In [ ]:
### Downsample Data in Batches of 100.000 cells for the grnboost2.

N_total = 100000

adata.obs["strata"] = adata.obs["Sample_ID"].astype(str) + "__" + adata.obs["Cluster_Column"].astype(str)

group_counts = adata.obs["strata"].value_counts()

sample_frac = N_total / len(adata)
sample_counts = (group_counts * sample_frac).round().astype(int)
sample_counts[sample_counts < 1] = 100

indices = []
for group, n in sample_counts.items():
    group_indices = adata.obs_names[adata.obs["strata"] == group]
    n = min(len(group_indices), n)
    sampled = np.random.choice(group_indices, size = n, replace = False)
    indices.extend(sampled)

adata_downsampled = adata[indices].copy()

print(f"✅ Downsampled to {adata_downsampled.n_obs:,} cells (from {adata.n_obs:,})")

adata_downsampled.obs.drop(columns = "strata", inplace = True)
adata_downsampled.write_h5ad("adata_downsampled_100k.h5ad")

In [ ]:
### Downsample Data in the top 5000/1000 hvg for the grnboost2.

adata_downsampled.X = adata_downsampled.layers['counts_RNA'].copy()
expr_genes = adata_downsampled.var_names.tolist()
sc.pp.highly_variable_genes(adata_downsampled, n_top_genes = 5000, flavor = 'seurat_v3')
hvg_genes = adata_downsampled.var.loc[adata_downsampled.var['highly_variable'], :].index.tolist()

os.chdir("/auxilliaries/")
with open("TFs_top_25pc.txt") as f:
    tf_list = [g.strip() for g in f]

grn_genes = list(set(hvg_genes).union(set(tf_list)).intersection(set(expr_genes)))

adata_grn = adata_downsampled[:, grn_genes].copy()

In [ ]:
### Downstream Analysis of the Subset Adata.

adata_grn.X = adata_grn.layers['counts_RNA'].copy()
adata_grn.X.max()
sc.pp.normalize_total(adata_grn, target_sum = 1e4)
adata_grn.layers["data"] = adata_grn.X.copy()
sc.pp.log1p(adata_grn)
adata_grn.layers["log1p_normalized"] = adata_grn.X.copy()
sc.pp.highly_variable_genes(adata_grn, n_top_genes = 5000, flavor = 'seurat')
sc.pp.scale(adata_grn, max_value = 10)
adata_grn.layers["scale.data"] = adata_grn.X.copy()

In [ ]:
### Save Expression Matrix.

os.chdir("/resources/")

expr_matrix = adata_sub.X.A if hasattr(adata_grn.X, "A") else adata_grn.X

df = pd.DataFrame(
    expr_matrix, 
    index = adata_grn.obs_names, 
    columns = adata_grn.var_names
)
#df_T = df.T

df.to_csv(
    "SCENIC_Downsampled_Normalized_Matrix_Cell_Gene.csv",
    index = True
)

print(f"✅ Saved normalized matrix with shape {df.shape} to SCENIC_Downsampled_Normalized_Matrix_Cell_Gene.csv")

In [ ]:
### STEP 1. Output: List of Adjacencies Between a TF and its Targets. 
# Run in the Downsampled Dataset.

### In CLI.

pyscenic grn \
--num_workers 35 \
--output /user/results/SCENIC_Adjacencies.tsv \
--method grnboost2 \
/user/resources/SCENIC_Downsampled_Normalized_Matrix_Cell_Gene.csv \
/user/auxilliaries/TFs_top_25pc.txt

In [ ]:
### STEP 2-3. The Results is a List of Enriched Motifs for the Modules.
# Run in the Whole Dataset.

### In CLI.

pyscenic ctx \
/user/results/SCENIC_Adjacencies.tsv \
/user/auxilliaries/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \
/user/auxilliaries/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \
--annotations_fname /user/auxilliaries/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl \
--expression_mtx_fname /user/resources/SCENIC_All_Normalized_Matrix_Cell_Gene.csv \
--mode "dask_multiprocessing" \
--output /user/results/SCENIC_Regulon.csv \
--num_workers 35
#--mask_dropouts

In [ ]:
## Load Motifs.

df_motifs = load_motifs(MOTIFS_FNAME)

In [ ]:
### Function: Derive Regulons.

def derive_regulons(motifs, db_names = ("hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings",
                                       "hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather")):
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype = bool) & \
        np.fromiter(map(contains(*db_names), motifs.Context), dtype = bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype = bool)]

    regulons = list(filter(lambda r: len(r) >= 5, df2regulons(motifs[(motifs['NES'] >= 2.0) ### Change NES
                                                                      & ((motifs['Annotation'] == 'gene is directly annotated')
                                                                        | (motifs['Annotation'].str.startswith('gene is orthologous to')
                                                                           & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
                                                                     ])))
              
    return list(map(lambda r: r.rename(r.transcription_factor), regulons))

regulons = derive_regulons(df_motifs)

In [ ]:
### STEP 4. Cellular Enrichment aka AUCell.

auc_mtx = aucell(df, regulons, num_workers = 35)
auc_mtx.to_csv(AUCELL_MTX_FNAME)

In [ ]:
### Add SCENIC Metadata to Adata Object.

add_scenic_metadata(adata_sub, auc_mtx, regulons)

In [ ]:
### OPTIONAL STEP 5 - Regulon Activity Binarization

bin_mtx, thresholds = binarize(auc_mtx)
bin_mtx.to_csv(BIN_MTX_FNAME) 
thresholds.to_frame().rename(columns = {0:'threshold'}).to_csv(THR_FNAME)

In [ ]:
### Read AUC, Bin Data and Thresholds.

auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col = 0)
bin_mtx = pd.read_csv(BIN_MTX_FNAME, index_col = 0)
thresholds = pd.read_csv(THR_FNAME, index_col = 0).threshold

In [ ]:
### Plots.

BASE_URL = "http://motifcollections.aertslab.org/v9/logos/"
COLUMN_NAME_LOGO = "MotifLogo"
COLUMN_NAME_MOTIF_ID = "MotifID"
COLUMN_NAME_TARGETS = "TargetGenes"

def display_logos(df: pd.DataFrame, top_target_genes: int = 3, base_url: str = BASE_URL):
    df = df.copy()
    def create_url(motif_id):
        return '<img src="{}{}.png" style="max-height:124px;"></img>'.format(base_url, motif_id)
    df[("Enrichment", COLUMN_NAME_LOGO)] = list(map(create_url, df.index.get_level_values(COLUMN_NAME_MOTIF_ID)))
    
    def truncate(col_val):
        return sorted(col_val, key=op.itemgetter(1))[:top_target_genes]
    df[("Enrichment", COLUMN_NAME_TARGETS)] = list(map(truncate, df[("Enrichment", COLUMN_NAME_TARGETS)]))
    
    MAX_COL_WIDTH = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', 200)
    display(HTML(df.head().to_html(escape = False)))
    pd.set_option('display.max_colwidth', MAX_COL_WIDTH)
    

display_logos( df_motifs.sort_values([('Enrichment','NES')], ascending = False).head(9))

In [ ]:
### Generate a Z-score for Each Regulon to Enable Comparison Between Regulons.

auc_mtx_Z = pd.DataFrame( index = auc_mtx.index )
for col in list(auc_mtx.columns):
    auc_mtx_Z[ col ] = ( auc_mtx[col] - auc_mtx[col].mean()) / auc_mtx[col].std(ddof = 0)
#auc_mtx_Z.sort_index(inplace=True)

In [ ]:
### Linear Mixed Model (LMM) For All Regulons.

def run_lmm_all_regulons(
        adata,
        cluster_key = "Cluster_Column",
        sample_key = "Sample_ID",
        reference_group = "Cluster_1"
):
    
    regulon_cols = [col for col in adata.obs.columns if col.startswith("Regulon(")]
    results = []

    for regulon in tqdm(regulon_cols, desc = "Fitting LMMs"):
        df = adata.obs[[regulon, cluster_key, sample_key]].copy()
        df.columns = ["AUC", "Cluster", "Sample"]

        if df["AUC"].nunique() < 2:
            continue

        df["AUC_trans"] = np.arcsin(np.sqrt(df["AUC"].clip(0, 1)))
        df["Cluster"] = df["Cluster"].astype("category")

        if reference_group not in df["Cluster"].cat.categories:
            continue

        cats = [reference_group] + [c for c in df["Cluster"].cat.categories if c != reference_group]
        df["Cluster"] = df["Cluster"].cat.reorder_categories(cats, ordered = True)

        try:
            model = smf.mixedlm("AUC_trans ~ C(Cluster)", df, groups = df["Sample"])
            res = model.fit(reml = True, method = "lbfgs", maxiter = 500, disp = False)

            for level in df["Cluster"].cat.categories[1:]:
                term = f"C(Cluster)[T.{level}]"
                results.append({
                    "Regulon": regulon,
                    "Cluster": level,
                    "EffectSize": res.params.get(term, np.nan),
                    "p_value": res.pvalues.get(term, np.nan)
                })

        except Exception as e:
            print(f"LMM failed for {regulon}: {e}")
            continue

    results_df = pd.DataFrame(results)
    results_df["absEffect"] = results_df["EffectSize"].abs()
    results_df["FDR"] = multipletests(results_df["p_value"], method = "fdr_bh")[1]
    results_df = results_df.sort_values(["absEffect", "p_value"], ascending = [False, True])
    
    return results_df

In [ ]:
### Linear Mixed Model (LMM) For a Single Regulon.

def fit_lmm_for_regulon(
        adata,
        regulon_col,
        cluster_key = "Cluster_Column",
        sample_key = "Sample_ID",
        reference_group = "Cluster_1",
        transform = "arcsin"
):
    
    df = adata.obs[[regulon_col, cluster_key, sample_key]].copy()
    df.columns = ["AUC", "Cluster", "Sample"]

    if transform == "arcsin":
        df["AUC_trans"] = np.arcsin(np.sqrt(df["AUC"]))
    elif transform == "logit":
        eps = 1e-4
        df["AUC_trans"] = np.log((df["AUC"] + eps) / (1 - df["AUC"] + eps))
    else:
        df["AUC_trans"] = df["AUC"]

    df["Cluster"] = df["Cluster"].astype("category")

    formula = f"AUC_trans ~ C(Cluster, Treatment(reference='{reference_group}'))"

    try:
        model = smf.mixedlm(formula, data = df, groups = df["Sample"])
        res = model.fit(reml = False)

        return pd.DataFrame({
            "Term": res.params.index,
            "Beta": res.params.values,
            "SE": res.bse.values,
            "p_value": res.pvalues.values,
            "Regulon": regulon_col
        })

    except Exception as e:
        print(f"LMM failed for {regulon_col}: {e}")
        return pd.DataFrame()

In [ ]:
### Network Plot of a Single Regulon.

def plot_regulon_network(regulon_obj, figsize = (8, 8), title = None):

    G = nx.DiGraph()
    tf = regulon_obj.name
    
    G.add_node(tf, type = "TF")
    for t in regulon_obj.genes:
        G.add_node(t, type = "Target")
        G.add_edge(tf, t)

    colors = ["red" if G.nodes[n]["type"] == "TF" else "skyblue" for n in G.nodes()]
    sizes = [800 if G.nodes[n]["type"] == "TF" else 300 for n in G.nodes()]

    pos = nx.spring_layout(G, seed = 42)
    
    plt.figure(figsize = figsize)
    nx.draw(G, pos, node_color = colors, node_size = sizes, with_labels = True, font_size = 7)
    plt.title(title if title else f"Regulon Network: {tf}")
    plt.show()

In [ ]:
### BoxPlot.

def plot_auc_boxplot(
        adata,
        regulon_col,
        cluster_key,
        clusters_of_interest,
        colors,
        figsize = (4, 4),
        filename=None
):
    
    df = adata.obs[[regulon_col, cluster_key]].copy()
    df.columns = ["AUC", "Cluster"]
    df["Cluster"] = df["Cluster"].astype(str)

    df = df[df["Cluster"].isin(clusters_of_interest)]

    sns.set(style = "white")
    fig, ax = plt.subplots(figsize = figsize)

    sns.boxplot(
        data = df,
        x = "Cluster",
        y = "AUC",
        order = clusters_of_interest,
        palette = colors,
        width = 0.6,
        fliersize = 0,
        linewidth = 0,
        ax = ax
    )

    for i, c in enumerate(clusters_of_interest):
        y = df[df["Cluster"] == c]["AUC"]
        x = np.random.normal(i, 0.08, len(y))
        ax.scatter(x, y, color=colors[i], alpha = 0.15, s = 8, edgecolor = "none")

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("")
    ax.set_ylabel("AUC", fontsize=10)
    ax.set_xticklabels([])

    plt.tight_layout()

    if filename:
        fig.savefig(filename, dpi = 800, bbox_inches = "tight")
    
    return fig

In [ ]:
### MatrixPlot For Regulon Activity.

def plot_regulon_matrix(adata, regulon_list, cluster_key, order, filename = None):
    
    regulon_clean = [re.sub(r"Regulon\((.*?)\)", r"\1", r) for r in regulon_list]

    ad_tmp = sc.AnnData(X = adata.obs[regulon_list].values, obs = adata.obs.copy())
    ad_tmp.var_names = regulon_clean

    matrix = sc.pl.matrixplot(
        ad_tmp,
        var_names = regulon_clean,
        groupby = cluster_key,
        categories_order = order,
        cmap = "Spectral_r",
        return_fig = True,
        show = False
    )

    if filename:
        matrix.fig.savefig(filename, dpi = 800, bbox_inches = "tight", transparent = True)

    return matrix

In [ ]:
### DotPlot of Target Genes of a Single Regulon.

marker_genes = ["Feature_1", "Feature_2", "Feature_3"
]

my_levels = [
    "Cluster_1", "Cluster_2", "Cluster_3", "Cluster_3"
]

adata_sub.obs["Cluster_Column"] = adata_sub.obs["Cluster_Column"].astype("category")
adata_sub.obs["Cluster_Column"] = adata_sub.obs["Cluster_Column"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Cluster_Column",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(4, 13)
dot.fig.tight_layout()
dot.fig.savefig("dotplot_targets_regulon.png", dpi = 800, bbox_inches = "tight")